In [ ]:
import os
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments, Trainer, pipeline
from peft import LoraConfig
from datasets import Dataset
import datasets
from trl import SFTTrainer, PPOTrainer
#load model name
# model_name = "qwen/Qwen2.5-0.5B"
model_name = "qwen/Qwen2.5-1.5B-Instruct"
# model_name = "bkai-foundation-models/vietnamese-llama2-7b-120GB"

In [ ]:

based_model = AutoModelForCausalLM.from_pretrained(model_name,
  # quantization_config=quant_config,
  torch_dtype=torch.float16,
  device_map={"":0}
)

config.json:   0%|          | 0.00/684 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/138 [00:00<?, ?B/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/7.23k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

In [ ]:
peft_params = LoraConfig(
    r=128,
    lora_alpha=16,
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM",
)

In [ ]:
gsm8k = datasets.load_dataset("gsm8k", "main")

In [ ]:
gsm8k_train, gsm8k_test = gsm8k['train'], gsm8k['test']

In [ ]:
from tqdm import tqdm

In [ ]:
def preprocess_function(examples):
    inputs = [f"<|im_start|>user\n{prompt}<|im_end|>\n" for prompt in examples["question"]]
    # print(inputs[0])
    targets = [f"<|im_start|>assistant\n{completion}<|im_end|>\n" for completion in examples["answer"]]
    # print(targets[0])
    # model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")
    model_inputs = tokenizer(inputs,
                            #  max_length=512, padding = True
                             )
    # labels = tokenizer(targets, max_length=512, truncation=True, padding = True)
    labels = tokenizer(targets,
                      #  max_length=512, padding = True
                       )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


In [ ]:
tokenized_dataset = gsm8k_train.map(preprocess_function, batched=True, remove_columns=gsm8k_train.column_names)

In [ ]:
tokenized_eval_dataset = gsm8k_test.map(preprocess_function, batched=True, remove_columns=gsm8k_test.column_names)

In [ ]:
# %cd /content/drive/MyDrive/qwen

/content/drive/MyDrive


In [ ]:
training_params = TrainingArguments(
    output_dir="./results",
    num_train_epochs=5,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=1,
    logging_steps=50,
    learning_rate=2e-4,
    logging_dir="./logs",
    save_strategy="epoch",
    fp16=True,
    optim="adamw_torch"
)

trainer = SFTTrainer(
    model=based_model,
    train_dataset=tokenized_dataset,
    eval_dataset=tokenized_eval_dataset,
    peft_config=peft_params,
    max_seq_length=None,
    args=training_params,
    packing=False,
)


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:309: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


In [ ]:
#turn off wandb
import os
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
# trainer.train() #This is 0.5B

In [ ]:
trainer.train() #This is 1.5B

In [ ]:
based_model

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 896)
    (layers): ModuleList(
      (0-23): 24 x Qwen2DecoderLayer(
        (self_attn): Qwen2SdpaAttention(
          (q_proj): lora.Linear(
            (base_layer): Linear(in_features=896, out_features=896, bias=True)
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.1, inplace=False)
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=896, out_features=128, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=128, out_features=896, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
            (lora_magnitude_vector): ModuleDict()
          )
          (k_proj): Linear(in_features=896, out_features=128, bias=True)
          (v_proj): lora.Linear(
            (base_layer): Linear(in_features=896, out_features=128, bias

In [ ]:
### in ra output tu 1 prompt
prompt = '''<|im_start|>user
Jen buys and sells candy bars. She buys candy bars for 80 cents each and sells them for a dollar each. If she buys 50 candy bars and sells 48 of them, how much profit does she make in cents?<|im_end|>

<|im_start|>assistant'''
output = based_model.generate(tokenizer(prompt, return_tensors='pt').input_ids.to('cuda'), early_stopping=True)


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:638: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


In [ ]:
### in ra output tu 1 prompt
prompt = '''<|im_start|>user
Jen buys and sells candy bars. She buys candy bars for 80 cents each and sells them for a dollar each. If she buys 50 candy bars and sells 48 of them, how much profit does she make in cents?<|im_end|>

<|im_start|>assistant'''
output = old_model.generate(tokenizer(prompt, return_tensors='pt').input_ids.to('cuda'), pad_token_id=tokenizer.eos_token_id, early_stopping=True, max_new_tokens=200)


In [ ]:
print(tokenizer.decode(output[0]))

<|im_start|>user
Jen buys and sells candy bars. She buys candy bars for 80 cents each and sells them for a dollar each. If she buys 50 candy bars and sells 48 of them, how much profit does she make in cents?<|im_end|>

<|im_start|>assistant
Jen buys 50 candy bars for 80 cents each, so she spends 50 * 80 = 4000 cents on candy bars.
She sells 48 candy bars for 100 cents each, so she earns 48 * 100 = 4800 cents from selling candy bars.
Her profit is her earnings minus the cost of buying the candy bars, so her profit is 4800 - 4000 = 800 cents.
#### 800
The answer is: 800<|endoftext|>
